In [1]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_batch


In [2]:
import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/recent"

headers = {
	"x-rapidapi-key": "51467c140emsh26c4d432d60126bp1d911djsn5deb38f38962",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

print(response.json())


{'typeMatches': [{'matchType': 'International', 'seriesMatches': [{'seriesAdWrapper': {'seriesId': 11253, 'seriesName': "ICC Men's T20 World Cup 2026", 'matches': [{'matchInfo': {'matchId': 139263, 'seriesId': 11253, 'seriesName': "ICC Men's T20 World Cup 2026", 'matchDesc': '32nd Match, Group B', 'matchFormat': 'T20', 'startDate': '1771320600000', 'endDate': '1771333200000', 'state': 'Complete', 'status': 'Match abandoned due to rain (no toss)', 'team1': {'teamId': 27, 'teamName': 'Ireland', 'teamSName': 'IRE', 'imageId': 839366}, 'team2': {'teamId': 12, 'teamName': 'Zimbabwe', 'teamSName': 'ZIM', 'imageId': 776198}, 'venueInfo': {'id': 254, 'ground': 'Pallekele International Cricket Stadium', 'city': 'Pallekele', 'timezone': '+05:30', 'latitude': '7.280691', 'longitude': '80.722493'}, 'seriesStartDt': '1770422400000', 'seriesEndDt': '1773100800000', 'isTimeAnnounced': True, 'stateTitle': 'Abandon', 'isFantasyEnabled': True}}, {'matchInfo': {'matchId': 139274, 'seriesId': 11253, 'seri

In [3]:
rec_20_data = response.json()

In [4]:
matches = []

for tm in rec_20_data['typeMatches']:
    for sm in tm.get('seriesMatches', []):
        series = sm.get('seriesAdWrapper', {})
        for match in series.get('matches', []):
            info = match.get('matchInfo', {})
            
            if info.get('state') != 'Complete':
                continue  # only completed matches

            matches.append({
                'match_desc': info.get('matchDesc'),
                'team1': info['team1']['teamName'],
                'team2': info['team2']['teamName'],
                'status': info.get('status'),
                'venue': info['venueInfo']['ground'],
                'start_date': int(info.get('startDate', 0))
            })


In [5]:
def parse_result(status):
    if 'won by' not in status:
        return None, None, None

    winner = status.split(' won by ')[0]
    margin_part = status.split(' won by ')[1]

    if 'runs' in margin_part:
        return winner, margin_part.replace(' runs', ''), 'runs'
    if 'wkts' in margin_part or 'wickets' in margin_part:
        return winner, margin_part.replace(' wkts', '').replace(' wickets', ''), 'wickets'

    return winner, margin_part, 'other'


In [6]:
for m in matches:
    winner, margin, victory_type = parse_result(m['status'])
    m['winning_team'] = winner
    m['victory_margin'] = margin
    m['victory_type'] = victory_type


In [7]:
recent_20 = sorted(
    matches,
    key=lambda x: x['start_date'],
    reverse=True
)[:20]


In [8]:
import pandas as pd

df = pd.DataFrame(recent_20)[[
    'match_desc',
    'team1',
    'team2',
    'winning_team',
    'victory_margin',
    'victory_type',
    'venue'
]]

df


,match_desc,team1,team2,winning_team,victory_margin,victory_type,venue
0,"33rd Match, Group C",Scotland,Nepal,Nepal,7,wickets,Wankhede Stadium
1,"32nd Match, Group B",Ireland,Zimbabwe,None,None,None,Pallekele International Cricket Stadium
2,"10th Match, Group A",United Arab Emirates Women,Pakistan A Women,Pakistan A Women,9,wickets,Terdthai Cricket Ground
3,"31st Match, Group D",Canada,New Zealand,New Zealand,8,wickets,MA Chidambaram Stadium
4,"9th Match, Group A",Nepal Women,India A Women,India A Women,7,wickets,Terdthai Cricket Ground
5,28th Match,Otago,Auckland,Auckland,6,wickets,John Davies Oval
6,29th Match,Central Districts,Canterbury,Central Districts,22,runs,Saxton Oval
7,3rd T20I,Pakistan Women,South Africa Women,Pakistan Women,53,runs,Diamond Oval
8,"30th Match, Group B",Australia,Sri Lanka,Sri Lanka,8,wickets,Pallekele International Cricket Stadium
9,"29th Match, Group C",England,Italy,England,24,runs,Eden Gardens


In [9]:
DB_CONFIG = {
    "host": "localhost",
    "database": "guviprojbackup",
    "user": "postgres",
    "password": "ssbv7792",
    "port": 5432
}

conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()
print("✅ Connected to PostgreSQL")

cur.execute("""
CREATE TABLE IF NOT EXISTS last_20_match_results (
    match_desc TEXT,
    team1 TEXT,
    team2 TEXT,
    winning_team TEXT,
    victory_margin INT,
    victory_type TEXT,
    venue TEXT
);
""")

conn.commit()
print("✅ Table created")

insert_query = """
INSERT INTO last_20_match_results (
    match_desc,
    team1,
    team2,
    winning_team,
    victory_margin,
    victory_type,
    venue
)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""
data_tuples = [
    (
        row["match_desc"],
        row["team1"],
        row["team2"],
        row["winning_team"],
        row["victory_margin"],
        row["victory_type"],
        row["venue"]
    )
    for _, row in df.iterrows()
]
execute_batch(cur, insert_query, data_tuples)
conn.commit()

print(f"✅ Inserted {len(data_tuples)} rows into match_results")
cur.close()
conn.close()
print("✅ Done")



✅ Connected to PostgreSQL
✅ Table created


InvalidTextRepresentation: invalid input syntax for type integer: "21 (DLS method)"
LINE 187: ... 'Australia Women', 'India Women', 'India Women', '21 (DLS m...
                                                               ^
